In [29]:
import requests
import json
import pandas as pd

# Define the API URL and parameters
api_url = "https://presto.lbl.gov/api/v1/model"
api_key = "4cc450bc-be38-4af1-9761-3e69a2263800"
fips = input("Enter FIPS code for region of interest")

# Define headers and data for the request
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
}

data = {
    "key": api_key,
    "fips": fips,
}

# Make the POST request
response = requests.post(api_url, headers=headers, data=json.dumps(data))

# Check if the request was successful (status code 200)
if response.status_code == 200:
    response_json = response.json()

    # Create a Pandas DataFrame from the response data
    df = pd.DataFrame(response_json["results"])

    # Expand the nested lists into separate DataFrames and concatenate them
    df_weeks = pd.DataFrame(df["weeks"].tolist(), columns=[f"week_{i}" for i in range(df["weeks"].apply(len).max())])
    df_days = pd.DataFrame(df["daysOfMonth"].tolist(), columns=[f"day_{i}" for i in range(df["daysOfMonth"].apply(len).max())])
    df_hours = pd.DataFrame(df["hoursOfDay"].tolist(), columns=[f"hour_{i}" for i in range(df["hoursOfDay"].apply(len).max())])
    df_durations = pd.DataFrame(df["durations"].tolist(), columns=[f"duration_{i}" for i in range(df["durations"].apply(len).max())])
    df_months = pd.DataFrame(df["months"].tolist(), columns=[f"month_{i}" for i in range(df["months"].apply(len).max())])

    # Concatenate the separate DataFrames
    df = pd.concat([df, df_weeks, df_days, df_hours, df_durations, df_months], axis=1)

    # Drop the original columns with nested lists
    df = df.drop(["weeks", "daysOfMonth", "hoursOfDay", "durations", "months"], axis=1)

    # Save the reformatted DataFrame to a CSV file
    df.to_csv("reformatted_api_data.csv", index=False)

    print("Reformatted data saved to 'reformatted_api_data.csv'.")
else:
    print("Failed to make the API request. Status code:", response.status_code)


Enter FIPS code for region of interest 08005


Reformatted data saved to 'reformatted_api_data.csv'.
